In [1]:
print("ok")

ok


In [3]:
import langchain  
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.vectorstores import Pinecone
import pinecone
from langchain_community.llms import CTransformers
from langchain_community.output_parsers.rail_parser import GuardrailsOutputParser
from langchain.chains import LLMChain, RetrievalQA
from langchain_core.prompts import PromptTemplate
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
# import pypdf

d:\GenAI\Medical-Chatbot-using-Llama\env\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()

    return documents

In [6]:
extract_data = load_pdf("D:\GenAI\Medical-Chatbot-using-Llama\data")

In [7]:
# Create text chunks

def text_split(extract_data):
    text_splitter  = RecursiveCharacterTextSplitter(chunk_size =500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extract_data)
    return text_chunks

text_chunks = text_split(extract_data)
print("lenght of my chunks",len(text_chunks))

lenght of my chunks 7020


In [8]:
print(text_chunks[1].page_content)

TheGALE
ENCYCLOPEDIA
ofMEDICINE
SECOND EDITION
JACQUELINE L. LONGE, EDITOR
DEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR
VOLUME
A-B1


In [22]:
# download embeddding model
def download_hugging_face_embeddings():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    # model_kwargs = {'device': 'gpu'}
    # encode_kwargs = {'normalize_embeddings': False}
    hf = HuggingFaceEmbeddings(
        model_name=model_name,
        # model_kwargs=model_kwargs,
        # encode_kwargs=encode_kwargs
    )
    
    return hf
embeddings = download_hugging_face_embeddings()

d:\GenAI\Medical-Chatbot-using-Llama\env\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [21]:
query_r = embeddings.embed_query("hello world")
len(query_r)

384

In [24]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY")
)

index_name = "medical-chatbot"

# pc.create_index(
#     name=index_name,
#     dimension=384,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud='aws',
#         region='us-east-1'
#     )
# )


# PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY", "bacf5bc5-9259-45f9-ab02-8bb403b40cbf")
# PINECONE_API_ENV = os.environ.get("PINECONE_API_ENV", "gcp-starter")




In [25]:

from langchain_pinecone import Pinecone as PC
# docs_chunks = [t.page_content for t in text_chunks]
# docsearch = PC.from_texts(
#     docs_chunks,
#     embeddings,
#     index_name=index_name
# )

docsearch = PC.from_existing_index(index_name,embeddings)

In [15]:
query = "what are Allergies"
docs = docsearch.similarity_search(query,k=3)
print("Result",docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly ca

In [16]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answers, just say that you don't know, don't try to make up an answer.

Context: {contest}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [17]:
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=['contest','question'],

)

chain_type_kwargs={"prompt":PROMPT}

In [18]:
retriever = docsearch.as_retriever(search_kwargs = {"k":2})

In [19]:
# repo_id = "meta-llama/Meta-Llama-3-8B"
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=512, temperature=0.5, token=HUGGINGFACEHUB_API_TOKEN
)


WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\Abdel\.cache\huggingface\token
Login successful


In [20]:
#create a chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, # for refinement
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    # prompt = PROMPT,
    # chain_type_kwargs=chain_type_kwargs
)

In [21]:

user_input = input(f"Input Prompt:")
result = qa_chain.invoke(user_input)
print("Response : ",result["result"])

Response :   A stroke occurs when an embolus, or blood clot, blocks the flow of blood and oxygen to the brain. This can cause severe damage or death to the brain tissue beyond the blockage. Symptoms may include sudden weakness or paralysis on one side of the body, difficulty speaking or understanding speech, and loss of vision in one or both eyes. It's important to note that strokes can also be caused by other factors, such as ruptured blood vessels in the brain.
